# Generating roughness blocks

The standard use case for generating the roughness blocks is to use a configuration file containing the necessary parameters.

These parameters account for the number of blocks being generated in each direction and also the offset and spacing applied to each row.

In [6]:
import pathlib
import pprint
from cfdmod.use_cases.block_gen import GenerationParams

pp = pprint.PrettyPrinter()

cfg_file = pathlib.Path("./fixtures/tests/block_gen/block_params.yaml")
cfg = GenerationParams.from_file(cfg_file)

pp.pprint(cfg.model_dump())

{'N_blocks_x': 10,
 'N_blocks_y': 10,
 'block_params': {'height': 5.0, 'length': 5.0, 'width': 5.0},
 'spacing_params': {'line_offset': 5.0,
                    'offset_direction': 'x',
                    'spacing': (2.0, 2.0)}}


It's also possible to generate the configurations through code.
This can be done by creating an instance of `GenerationParams` and using it for the next steps.

The example above can be instanciated as

In [7]:
from cfdmod.use_cases.block_gen import SpacingParams, BlockParams

block_params = BlockParams(height=5, width=5, length=5)

spacing_params = SpacingParams(spacing=(2, 2), line_offset=5, offset_direction="x")

cfg = GenerationParams(
    N_blocks_x=10, N_blocks_y=10, block_params=block_params, spacing_params=spacing_params
)

## Build block and linear patterns

In order to build all the blocks, linear patterns are applied.
The first one applies to create a row in the offset direction.
The second one applies to replicate the original row and apply an offset to odd index rows.

In [8]:
from cfdmod.use_cases.block_gen import build_single_block, linear_pattern

triangles, normals = build_single_block(cfg.block_params)

single_line_triangles, single_line_normals = linear_pattern(
    triangles,
    normals,
    direction=cfg.spacing_params.offset_direction,
    n_repeats=cfg.single_line_blocks,
    spacing_value=cfg.single_line_spacing,
)

full_triangles, full_normals = linear_pattern(
    single_line_triangles,
    single_line_normals,
    direction=cfg.perpendicular_direction,
    n_repeats=cfg.multi_line_blocks,
    spacing_value=cfg.multi_line_spacing,
    offset_value=cfg.spacing_params.line_offset,
)

print(
    "Single block:",
    f"Vertices count: {triangles.shape[0] * triangles.shape[1]}",
    f"Triangles count: {len(triangles)}",
)
print(
    "Single line",
    f"Vertices count: {single_line_triangles.shape[0] * single_line_triangles.shape[1]}",
    f"Triangles count: {len(single_line_triangles)}",
)
print(
    "Replicated lines",
    f"Vertices count: {full_triangles.shape[0] * full_triangles.shape[1]}",
    f"Triangles count: {len(full_triangles)}",
)

Single block: Vertices count: 30 Triangles count: 10
Single line Vertices count: 300 Triangles count: 100
Replicated lines Vertices count: 3000 Triangles count: 1000


## Export generated geometry

In order to use the generated geometry, the user must export the triangles and normals, using the geometry API and the STL format (triangles, normals).

In [9]:
from cfdmod.api.geometry.STL import export_stl
import pathlib

output_path = pathlib.Path("./output/block_gen")

export_stl(output_path / "blocks.stl", full_triangles, full_normals)